<a href="https://colab.research.google.com/github/CharuniPremarathne/Assignment_DL/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [11]:
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import random

Mount Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!cp /content/drive/MyDrive/RP/ARMD_NEW.zip /content

In [7]:
!mkdir /content/ARMD
!unzip -q ARMD_NEW.zip -d /content/ARMD/

Loading the dataset

In [12]:
image_files = [f for f in glob.glob(r'/content/ARMD/ARMD.v2i.tensorflow/train/**/*.jpg', recursive=True) if not os.path.isdir(f)]

Define Image Dimensions

In [13]:
image_dimensions = (224, 224, 3)

Creating empty list

In [14]:
data = []
labels = []

Convert the array and lavel the catrgories

In [15]:
for img in image_files:
  image = cv2.imread(img)
  image = cv2.resize(image, (image_dimensions[0], image_dimensions[1]))
  image = img_to_array(image)
  data.append(image)
  label = img.split(os.path.sep)[-2]
  if label == "ARMD":
    label = 1
  else:
    label = 0
  labels.append([label])

In [16]:
print(labels)

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1],

Convert to numpy array

In [17]:
data = np.array(data, dtype = "float")

In [19]:
labels = np.array(labels)

TRain & Test split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state=42)

Expanding the dimensions

In [21]:
trainy = to_categorical(y_train, num_classes = 2)
testy = to_categorical(y_test, num_classes = 2)

Train the VGG16 model

In [22]:
vgg16 = VGG16(input_shape=(224, 224, 3), weights="imagenet", include_top = False)

58889256/58889256 [==============================] - 0s 0us/step


In [23]:
for layers in vgg16.layers:
  layers.trainable = False

In [24]:
X = Flatten()(vgg16.output)

In [26]:
folders = glob.glob(r'/content/ARMD/ARMD.v2i.tensorflow/train/*')
len(folders)

2

In [2]:
prediction = Dense(len(folders), activation='softmax')(X)

model = Model(inputs=vgg16.input, outputs=prediction)

NameError: ignored

In [29]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
batch_size = 32

In [1]:
history=model.fit(X_train, trainy, batch_size=batch_size, validation_data=(X_test, testy), epochs=50)


NameError: ignored

In [ ]:
model.save('model.h5')

Plot the acccuracy and the loss plot

In [ ]:
plt.style.use("ggplot")
plt.figure()
epochs=100
N=epochs
plt.plot(np.arange(0, N), history.history['loss'], label='train_loss')
plt.plot(np.arange(0, N), history.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, N), history.history['accuracy'], label='train_acc')
plt.plot(np.arange(0, N), history.history['val_accuracy'], label='val_acc')

plt.title('Training loss and Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend(loc='upper right')

Testing on a sample image

In [ ]:
image = cv2.imread('/content/njn')

In [ ]:
img_resize = cv2.resize(image,(224,224))

In [ ]:
conf = model.pedict(img_resize)[0]

In [ ]:
classes = ['ARMD', 'Normal']

In [ ]:
idx = np.argmax(conf)
label=classes[idx]

In [ ]:
print(labels)